In [1]:
 import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from pathlib import Path
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from lockdowndates.core import LockdownDates
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates

from catboost import CatBoostRegressor
import datetime
from datetime import timedelta, date

In [2]:
X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

In [18]:
def prepro(X):
    ext_df = pd.read_csv('external_data/external_data.csv')
    ext_df['date'] = pd.to_datetime(ext_df['date'].values.astype('<M8[us]'))
    data = utils.is_jour_ferie(
    utils.is_lockdown(
        utils.create_x_weather(
            utils.apply_couvre_feu(
                utils.filter_columns(X)
            )
        )
    )
    )
    result = utils._encode_dates(data).drop(columns=['date'])
    return result


In [19]:
prepro(X)

,counter_id,latitude,longitude,is_couvre_feu,temp,precip,windspeed,visibility,icon_encoded,is_lockdown,is_ferie,is_vacances,year,month,day,weekday,hour,is_weekend
0,100049407-353255860,48.840801,2.333233,0,16.1,0.00,13.6,25.1,3,0,0,0,2020,9,1,1,1,0
1,100049407-353255859,48.840801,2.333233,0,16.1,0.00,13.6,25.1,3,0,0,0,2020,9,1,1,1,0
2,100036719-104036719,48.853720,2.357020,0,16.1,0.00,13.6,25.1,3,0,0,0,2020,9,1,1,1,0
3,100036719-103036719,48.853720,2.357020,0,16.1,0.00,13.6,25.1,3,0,0,0,2020,9,1,1,1,0
4,100063175-353277233,48.885290,2.326660,0,16.1,0.00,13.6,25.1,3,0,0,0,2020,9,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534976,100057329-103057329,48.842010,2.367290,0,22.1,2.54,16.6,13.7,1,0,0,0,2021,9,9,3,23,0
534977,100057380-104057380,48.864620,2.314440,0,22.1,2.54,16.6,13.7,1,0,0,0,2021,9,9,3,23,0
534978,100057380-103057380,48.864620,2.314440,0,22.1,2.54,16.6,13.7,1,0,0,0,2021,9,9,3,23,0
534979,100042374-110042374,48.848400,2.275860,0,22.1,2.54,16.6,13.7,1,0,0,0,2021,9,9,3,23,0


## CATBOOST

In [20]:
X, y = utils.get_train_data()

X_train, y_train, X_test, y_test = utils.train_test_split_temporal(X, y, delta_threshold="30 days")

num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']

preprocessing = FunctionTransformer(prepro)

col_transformer = ColumnTransformer(
    transformers=[
        
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(), cat_features)
    ],
    remainder='passthrough'
)

cat = CatBoostRegressor(
    depth=10,
    iterations=1500,
    rsm=0.25,
    sampling_frequency="PerTree",
    subsample=0.7,
    verbose=0,
)

pipe = make_pipeline(
    preprocessing,
    col_transformer,
    cat
)

pipe.fit(X_train, y_train)

predictions = pipe.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 0.43214222059156354


**SCORE**

iter 1500 : 0.43214222059156354 

1200 : 0.47

1400: 0.449

1600: 0.46

iter1500 with icon oh encoded: 0.4503845244145301


In [25]:
pipe.fit(X,y)

submission = pipe.predict(X_final_test)
pd.Series(submission).to_frame().rename_axis('Id').rename(columns={0:'log_bike_count'}).to_csv('submission22_iter1500_91223.csv')

## XGBoost

In [8]:
X, y = utils.get_train_data()

X_train, y_train, X_test, y_test = utils.train_test_split_temporal(X, y, delta_threshold="30 days")

num_features = ['temp', 'precip', 'windspeed', 'visibility','latitude','longitude']
cat_features = ['counter_id', 'hour','weekday','day','month']

preprocessing = FunctionTransformer(prepro)

col_transformer = ColumnTransformer(
    transformers=[
        
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(), cat_features)
    ],
    remainder='passthrough'  # This keeps all other columns unchanged
)




model = xgb.XGBRegressor(colsample_bynode=0.6,
        colsample_bytree=0.6,
        colsample_bylevel=0.5,
        learning_rate=0.1,
        n_estimators=400,
        max_depth=10,
        subsample=0.8)


pipe = make_pipeline(
    preprocessing,
    col_transformer,
    model
)

pipe.fit(X_train, y_train)

predictions = pipe.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 0.46590921407233993


In [ ]:
pipe.fit(X,y)

submission = pipe.predict(X_final_test)
pd.Series(submission).to_frame().rename_axis('Id').rename(columns={0:'log_bike_count'}).to_csv('submission20_91223.csv')

In [12]:
pipe.fit(X,y)

submission = pipe.predict(X_final_test)
pd.Series(submission).to_frame().rename_axis('Id').rename(columns={0:'log_bike_count'}).to_csv('submission21_iter2000_91223.csv')

In [78]:
def is_lockdown(X, date_column='date'):
    # Définir les plages de dates à vérifier (sans tenir compte des heures)
    date_ranges = [
        {"start": datetime.datetime(2020, 3, 17), "end": datetime.datetime(2020, 5, 11)},
        {"start": datetime.datetime(2020, 10, 30), "end": datetime.datetime(2020, 12, 15)},
        {"start": datetime.datetime(2021, 4, 3), "end": datetime.datetime(2021, 5, 3)}
    ]

    # Fonction pour vérifier si une date est dans l'un des intervalles
    def is_date_in_range(date):
        if not isinstance(date, str):
            date = date.strftime('%Y-%m-%d')

        dt = datetime.datetime.strptime(date, '%Y-%m-%d')
        for period in date_ranges:
            if period["start"].date() <= dt.date() <= period["end"].date():
                return 1
        return 0

    # Appliquer la fonction à la colonne 'date' et créer une nouvelle colonne pour le résultat
    X['is_lockdown'] = X[date_column].apply(is_date_in_range)

    return X


apply_date_range_check(X)

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,is_in_range,is_lockdown
400125,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233,0,0
408305,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233,0,0
87516,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,"48.85372,2.35702",Y2H19027732,48.853720,2.357020,0,0
98518,100036719-103036719,18 quai de l'Hôtel de Ville SE-NO,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,"48.85372,2.35702",Y2H19027732,48.853720,2.357020,0,0
875137,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,2020-09-01 01:00:00,2020-07-22,"48.88529,2.32666",Y2H20073268,48.885290,2.326660,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
792857,100057329-103057329,Totem 85 quai d'Austerlitz SE-NO,100057329,Totem 85 quai d'Austerlitz,2021-09-09 23:00:00,2020-02-18,"48.84201,2.36729",YTH19111508,48.842010,2.367290,0,0
805182,100057380-104057380,Totem Cours la Reine E-O,100057380,Totem Cours la Reine,2021-09-09 23:00:00,2020-02-11,"48.86462,2.31444",YTH19111509,48.864620,2.314440,0,0
815218,100057380-103057380,Totem Cours la Reine O-E,100057380,Totem Cours la Reine,2021-09-09 23:00:00,2020-02-11,"48.86462,2.31444",YTH19111509,48.864620,2.314440,0,0
125979,100042374-110042374,Voie Georges Pompidou NE-SO,100042374,Voie Georges Pompidou,2021-09-09 23:00:00,2017-12-15,"48.8484,2.27586",Y2H21025335,48.848400,2.275860,0,0


## EXT Meteo Data

In [37]:
ext_df = pd.read_csv('external_data/external_data.csv')
ext_df['date'] = pd.to_datetime(ext_df['date'].values.astype('<M8[us]'))
ext_df.columns

Index(['numer_sta', 'date', 'pmer', 'tend', 'cod_tend', 'dd', 'ff', 't', 'td',
       'u', 'vv', 'ww', 'w1', 'w2', 'n', 'nbas', 'hbas', 'cl', 'cm', 'ch',
       'pres', 'niv_bar', 'geop', 'tend24', 'tn12', 'tn24', 'tx12', 'tx24',
       'tminsol', 'sw', 'tw', 'raf10', 'rafper', 'per', 'etat_sol', 'ht_neige',
       'ssfrai', 'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24', 'phenspe1',
       'phenspe2', 'phenspe3', 'phenspe4', 'nnuage1', 'ctype1', 'hnuage1',
       'nnuage2', 'ctype2', 'hnuage2', 'nnuage3', 'ctype3', 'hnuage3',
       'nnuage4', 'ctype4', 'hnuage4'],
      dtype='object')

In [60]:
test = X.merge(ext_df[['date','ww']], how='left', on='date', suffixes={'','_drop'}).interpolate(method='backfill')

In [64]:
test['ww'].fillna(method='ffill')

0         2.0
1         2.0
2         2.0
3         2.0
4         2.0
         ... 
496876    3.0
496877    3.0
496878    3.0
496879    3.0
496880    3.0
Name: ww, Length: 496881, dtype: float64

In [48]:
ext_df[['date','pmer','tend','cod_tend','ww']][ext_df['date']=='2020-09-01 6:00:00']

,date,pmer,tend,cod_tend,ww
3084,2020-09-01 06:00:00,102000,10,3,3
